In [ ]:
!pip install sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import pickle
import os
import numpy as np
import cv2

In [ ]:
import sys
sys.path.append('/opt/workspace/src/python_scripts/')
import ops_files_operations as files
import ops_logger as log
import ops_face_detection as fd

In [ ]:
script_name = os.path.basename(__name__)
path_arr = os.getcwd().split(os.path.sep)
os.chdir(os.path.sep.join(path_arr[:path_arr.index('src')+1]))

In [ ]:
logger = log.create_logger('./models/logs',script_name,level='error')

In [ ]:
pickle_file = './models/embeddings/embeddings.pickle'
data = files.read_pickle_file(pickle_file,logger)

In [ ]:
le = LabelEncoder()
labels = le.fit_transform(data["names"])
embeddings = np.squeeze(np.array(data["embeddings"]))
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

In [ ]:
recognizer = SVC(C=1.0, kernel="linear", probability=True)
recognizer.fit(embeddings, labels)

In [ ]:
protoPath = './models/face_detector/deploy.prototxt'
modelPath = './models/face_detector/res10_300x300_ssd_iter_140000.caffemodel'
embeddingPath = './models/face_detector/openface.nn4.small2.v1.t7'
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
embedder = cv2.dnn.readNetFromTorch(embeddingPath)
test_image_path = './datasets/actor_faces/2037_cillian_murphy/cillian_murphy_7.jpg'
test_embeddings = fd.get_embeddings_from_image(test_image_path,detector,embedder,logger)

In [ ]:
test_array = np.array(test_embeddings)

In [ ]:
preds = recognizer.predict_proba(test_array)[0]
j = np.argmax(preds)
proba = preds[j]
name = le.classes_[j]

In [ ]:
name

In [ ]:
proba

In [ ]:
preds

In [ ]:
le_name_mapping